<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Insect Model

_by Jeremy Freeman (CarbonPlan), October 26, 2020_

This notebook demos insect modelling.


In [196]:
import numpy as np
import pandas as pd
from carbonplan_forests import setup, plot, load, fit, utils, prepare

In [197]:
setup.plotting(remote=False)

In [198]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load the raw FIA data grouped by repeated inventories


In [4]:
# df = load.fia(store="local", states="conus", group_repeats=True)
# df = load.terraclim(
#     store="local",
#     tlim=(int(df["year_0"].min()), 2020),
#     data_vars=["ppt", "tavg", "pdsi", "cwd", "pet", "vpd"],
#     data_aggs=["sum", "mean", "mean", "sum", "mean", "mean"],
#     df=df,
#     group_repeats=True,
# )

In [199]:
df = pd.read_csv(
    "/Users/freeman/Dropbox (Personal)/fia_wide_with_terraclim_v5.csv",
    low_memory=False,
)

Prepare data for model fitting

In [203]:
x, y, meta = prepare.insects(df)

In [204]:
x_z, x_mean, x_std = utils.zscore_2d(x)

Fit models

In [206]:
pf = meta.copy()
pf["observed"] = y
pf["predicted"] = np.NaN

models = {}
for code in pf["type_code"].unique():
    inds = pf["type_code"] == code
    if (y[inds].sum() > 1) & (y[inds].sum() > 1):
        model = fit.hurdle(x=x_z[inds], y=y[inds])
        models[code] = model
        pf.loc[inds, "predicted"] = model.predict(x=x_z[inds])

pf = pf[pf["predicted"] < 1]

Compute simple score metrics (on all data and just on the values where fraction insects was positive)

In [207]:
pf[["observed", "predicted"]].corr().iloc[0, 1] ** 2

0.14617618226139906

In [208]:
pf[pf["observed"] > 0][["observed", "predicted"]].corr().iloc[0, 1] ** 2

0.021004253476884824

Plot observed vs predicted

In [210]:
plot.xy(
    x=pf["observed"],
    y=pf["predicted"],
    xlabel="measured fraction insects",
    ylabel="predicted fraction insects",
    xlim=(0, 1),
    ylim=(0, 1),
    width=300,
    height=300,
    opacity=0.1,
)

alt.Chart(...)

Make a map of actual fraction insects


In [211]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["observed"],
    cmap="reds",
    clim=(0, 0.2),
    size=5,
    opacity=0.75,
)

alt.LayerChart(...)

Make a map of predicted fraction insects

In [214]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["predicted"],
    cmap="reds",
    clim=(0, 0.2),
    size=5,
    opacity=0.75,
)

alt.LayerChart(...)